## Анализ текущего уровня потребительской лояльности по результатам NPS-опросов

**Описание проекта**

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

Чтобы оценить результаты опроса, оценки обычно делят на три группы:

- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 
Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса. 

## Шаг 1. Подключение к базе

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine

pth1 = 'telecomm_csi.db'
pth2 = '/datasets/telecomm_csi.db'

if os.path.exists(pth1):
    path_to_db = pth1
elif os.path.exists(pth2):
    path_to_db = pth2
else:
    print('Something is wrong')

engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

query = """
            SELECT u.user_id, 
                    u.lt_day,
                    CASE
                        WHEN u.lt_day <= 365 THEN "new"
                        ELSE "old"
                    END AS is_new,
                    u.age,
                    CASE 
                        WHEN u.gender_segment == 1 THEN "женщина"
                        WHEN u.gender_segment == 0 THEN  "мужчина"
                    END AS gender_segment,
                    u.os_name,
                    u.cpe_type_name,
                    l.country,
                    l.city,
                    SUBSTR(a_g.title, 3) AS age_segment,
                    SUBSTR(t_s.title, 3) AS traffic_segment,
                    SUBSTR(lt_s.title, 3) AS lifetime_segment,
                    u.nps_score,
                    CASE
                        WHEN u.nps_score >= 9 THEN "promoters"
                        WHEN u.nps_score == 7 OR u.nps_score == 8 THEN "passives"
                        ELSE "detractors"
                    END AS nps_group
            FROM
                user AS u LEFT JOIN location AS l ON u.location_id = l.location_id
                LEFT JOIN age_segment AS a_g ON u.age_gr_id = a_g.age_gr_id
                LEFT JOIN traffic_segment AS t_s ON u.tr_gr_id = t_s.tr_gr_id
                LEFT JOIN lifetime_segment AS lt_s ON u.lt_gr_id = lt_s.lt_gr_id
                WHERE age IS NOT NULL AND gender_segment IS NOT NULL;
"""


df = pd.read_sql(query, engine)
df.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,old,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,old,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,old,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,old,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters


Запишем таблицу в файл `df.csv`

In [2]:
df.to_csv('df.csv', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501152 entries, 0 to 501151
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           501152 non-null  object 
 1   lt_day            501152 non-null  int64  
 2   is_new            501152 non-null  object 
 3   age               501152 non-null  float64
 4   gender_segment    501152 non-null  object 
 5   os_name           501152 non-null  object 
 6   cpe_type_name     501152 non-null  object 
 7   country           501152 non-null  object 
 8   city              501152 non-null  object 
 9   age_segment       501152 non-null  object 
 10  traffic_segment   501152 non-null  object 
 11  lifetime_segment  501152 non-null  object 
 12  nps_score         501152 non-null  int64  
 13  nps_group         501152 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.5+ MB


#### Шаг 3. Создание дашборда в Tableau

Построить дашборд, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков. 


[ССЫЛКА на дашборд](https://public.tableau.com/views/Book1_16722492160430/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)



#### Шаг 4. Ответить на вопросы с помощью дашборда в виде презентации. 
[ССЫЛКА на презентацию](https://docs.google.com/presentation/d/1OOOxWGsZfRNUpX504TNcygN0dtN0D_8zNK7MK8VE7Js/edit?usp=sharing)